In [7]:
import os
import dotenv
dotenv.load_dotenv()

True

In [5]:
from fastapi import FastAPI
from pydantic import BaseModel
from typing import List
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.prebuilt import create_react_agent
from langchain_groq import ChatGroq

In [8]:
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
os.environ["TAVILY_API_KEY"] = "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"

In [ ]:
MODEL_NAMES = [
    
]

In [9]:
tool_tavily = TavilySearchResults(max_results=2)

In [10]:
tools = [tool_tavily,]

FastAPI application

In [11]:
app = FastAPI(title="LangGraph AI Agent")

In [12]:
class RequestState(BaseModel):
    model_name : str
    system_prompt : str
    messages : List[str]

In [13]:
#Define an endpoint for handling chat requests
@app.post("/chat")
def chat_endpoint(request: RequestState):
    """API endpoint to interact with the chatbot using
    Langraph and tools.
    """
    if request.model not in MODEL_NAMES:
        return {"error" : "Invalid model name."}
    llm = ChatGroq(GROQ_API_KEY=GROQ_API_KEY, model=request.model)
    agent = create_react_agent(llm, tools=tools, state_modifier=request.system_prompt)
    state = {"messages": request.messages}
    result = agent.invoke(state)
    return result

In [14]:
if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host='127.0.0.1', port=8000)

RuntimeError: asyncio.run() cannot be called from a running event loop